# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [1]:
from xml.etree import ElementTree as ET
from xml.etree.ElementTree import Element
from xml.etree.ElementTree import SubElement
import pandas as pd
import numpy as np
from pandas import DataFrame

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [2]:
document_tree = ET.parse( './data/mondial_database_less.xml' )

In [3]:
# print names of all countries
for child in document_tree.getroot():
    print child.find('name').text

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [5]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print '* ' + element.find('name').text + ':',
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print capitals_string[:-2]

* Albania: Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece: Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia: Skopje, Kumanovo
* Serbia: Beograd, Novi Sad, Niš
* Montenegro: Podgorica
* Kosovo: Prishtine
* Andorra: Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [2]:
document = ET.parse( './data/mondial_database.xml' )

In [3]:
document

In [4]:
#10 countries with the lowest mortality rate
df = DataFrame(columns=('Country', 'Mortality'))
cnt = 0#Counter for indexing
for child in document.getroot():
    infant1 = child.find('infant_mortality')
    if infant1 is not None:
        rate = float(child.find('infant_mortality').text)
        countryName = child.find('name').text
        df.loc[cnt] = [countryName,rate]
        cnt = cnt + 1

df.sort(['Mortality'])[:10]



,Country,Mortality
36,Monaco,1.81
90,Japan,2.13
109,Bermuda,2.48
34,Norway,2.48
98,Singapore,2.53
35,Sweden,2.60
8,Czech Republic,2.63
72,Hong Kong,2.73
73,Macao,3.13
39,Iceland,3.15


In [292]:
#10 cities with the largest population
df1 = DataFrame(columns=('City', 'Population'))
cnt1 = 0#Counter for indexing
for group in document.findall( 'country/province/city' ):
    maxval = 0
    mycity = ''
    for node in group.getchildren():
        if node.tag == 'name':
            #print 'city:', node.text
            mycity = node.text
            
        if node.tag == 'population':
            if node.attrib.get('measured') == 'census':
                #print int(node.text)
                if int(node.text) >= maxval:
                    maxval = int(node.text) 
    #print mycity,maxval
    df1.loc[cnt1] = [mycity,maxval]
    cnt1 = cnt1+1
df1.sort(['Population'],ascending=False)[:10]

,City,Population
1278,Shanghai,22315474
1510,Delhi,12877470
1455,Mumbai,12442373
1277,Beijing,11716620
458,Moscow,11612885
2490,São Paulo,11152344
1279,Tianjin,11090314
1001,Guangzhou,11071424
1004,Shenzhen,10358381
1074,Wuhan,9785388


In [77]:
#10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
df2 = DataFrame(columns=('EthnicGroup', 'Percentage'))
cnt2 = 0
for group in document.findall( 'country' ):
    for node in group.getchildren():
        if node.tag == 'ethnicgroup':
             if node.attrib['percentage'] is not None:
            #print node.text,float(node.attrib['percentage'])/100
                rVal = float(node.attrib['percentage'])/100
                df2.loc[cnt2] = [node.text,rVal]
                cnt2 = cnt2+1
g2 = df2.groupby('EthnicGroup',sort=False).sum()
#g2.sort(['Pecentage'],ascending=False)[:10]
g2[:10]

,Percentage
EthnicGroup,
Albanian,2.1720
Greek,1.7300
Macedonian,0.6670
Turkish,1.2084
Gypsy,0.0270
Serb,1.2520
Montenegrin,0.4390
Hungarian,1.1350
Roma,0.1220


In [72]:
#4.name and country of a) longest river, b) largest lake and c) airport at highest elevation
df3 = DataFrame(columns=('Country', 'River','Length'))
cnt3 = 0
for group in document.findall( 'river' ):
    rCountry = group.attrib['country']
    for node in group.getchildren():
        if node.tag == 'name':
            rName = node.text
        if node.tag == 'length':
             if node.text is not None:
                rLength = float(node.text)
    df3.loc[cnt3] = [rCountry,rName,rLength]
    cnt3 = cnt3+1
df3.sort(['Length'],ascending=False)[:10]



,Country,River,Length
174,CO BR PE,Amazonas,6448
137,CN,Jangtse,6380
136,CN,Hwangho,4845
123,R,Lena,4400
205,RCB ZRE,Zaire,4374
138,CN LAO THA K VN,Mekong,4350
115,R KAZ CN,Irtysch,4248
186,RMM RN WAN RG,Niger,4184
160,USA,Missouri,4130
119,R,Jenissej,4092


In [70]:
df4 = DataFrame(columns=('Country', 'Lake','Area'))
cnt3 = 0
for group in document.findall( 'lake' ):
    rCountry = group.attrib['country']
    for node in group.getchildren():
        if node.tag == 'name':
            rName = node.text
        if node.tag == 'area':
             if node.text is not None:
                    rLength = float(node.text)
    df4.loc[cnt3] = [rCountry,rName,rLength]
    cnt3 = cnt3+1
df4.sort(['Area'],ascending=False)[:10]

,Country,Lake,Area
54,R AZ KAZ IR TM,Caspian Sea,386400
109,CDN USA,Lake Superior,82103
81,EAT EAK EAU,Lake Victoria,68870
106,CDN USA,Lake Huron,59600
108,USA,Lake Michigan,57800
47,IL JOR WEST,Dead Sea,41650
83,ZRE Z BI EAT,Lake Tanganjika,32893
98,CDN,Great Bear Lake,31792
43,R,Ozero Baikal,31492
89,MW MOC EAT,Lake Malawi,29600


In [69]:
df5 = DataFrame(columns=('Country', 'Airport','Elevation'))
cnt3 = 0
for group in document.findall( 'airport' ):
    rCountry = group.attrib['country']
    for node in group.getchildren():
        if node.tag == 'name':
            rName = node.text
        if node.tag == 'elevation':
            if node.text is not None:
                rLength = float(node.text)
    df5.loc[cnt3] = [rCountry,rName,rLength]
    cnt3 = cnt3+1
df5.sort_index(by = 'Elevation',ascending=False)[:10]


,Country,Airport,Elevation
80,BOL,El Alto Intl,4063
219,CN,Lhasa-Gonggar,4005
241,CN,Yushu Batang,3963
242,CN,Anqing Airport,3963
243,CN,Changde Airport,3963
813,PE,Juliaca,3827
815,PE,Teniente Alejandro Velasco Astete Intl,3311
82,BOL,Juana Azurduy De Padilla,2905
334,EC,Mariscal Sucre Intl,2813
805,PE,Coronel Fap Alfredo Mendivil Duarte,2719
